# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment, Run
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.runconfig import RunConfiguration
from azureml.core import ScriptRunConfig, Environment
from azureml.widgets import RunDetails
from azureml.core.dataset import Dataset
from sklearn.model_selection import train_test_split
from azureml.train.automl import AutoMLConfig
# get workspace
ws = Workspace.from_config()
import joblib # for saving best fit model
import requests
import json


## Dataset

### Overview
This data includes 82 columns for 2,930 properties in Ames, IA. The goal is to predict the cost of the house based on features of the given property.



In [2]:
ws = Workspace.from_config()

# Create experiment.
experiment_name = 'predict-housing-prices'

experiment=Experiment(ws, experiment_name)

# Get data
key = 'AmesHousingData'
if key in ws.datasets.keys():
    data = ws.datasets[key]
    print(f"Dataset {key} located and loaded in.")
else:
    print(f"Cannot find {key}.")

# Load to pandas df
df = data.to_pandas_dataframe()

Dataset AmesHousingData located and loaded in.


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2930 entries, 0 to 2929
Data columns (total 82 columns):
Order              2930 non-null int64
PID                2930 non-null int64
MS SubClass        2930 non-null int64
MS Zoning          2930 non-null object
Lot Frontage       2440 non-null float64
Lot Area           2930 non-null int64
Street             2930 non-null object
Alley              2930 non-null object
Lot Shape          2930 non-null object
Land Contour       2930 non-null object
Utilities          2930 non-null object
Lot Config         2930 non-null object
Land Slope         2930 non-null object
Neighborhood       2930 non-null object
Condition 1        2930 non-null object
Condition 2        2930 non-null object
Bldg Type          2930 non-null object
House Style        2930 non-null object
Overall Qual       2930 non-null int64
Overall Cond       2930 non-null int64
Year Built         2930 non-null int64
Year Remod/Add     2930 non-null int64
Roof Style         2

In [4]:
df.describe()

,Order,PID,MS SubClass,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,...,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,SalePrice
count,2930.00000,2.930000e+03,2930.000000,2440.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2907.000000,...,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000
mean,1465.50000,7.144645e+08,57.387372,69.224590,10147.921843,6.094881,5.563140,1971.356314,1984.266553,101.896801,...,93.751877,47.533447,23.011604,2.592491,16.002048,2.243345,50.635154,6.216041,2007.790444,180796.060068
std,845.96247,1.887308e+08,42.638025,23.365335,7880.017759,1.411026,1.111537,30.245361,20.860286,179.112611,...,126.361562,67.483400,64.139059,25.141331,56.087370,35.597181,566.344288,2.714492,1.316613,79886.692357
min,1.00000,5.263011e+08,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,12789.000000
25%,733.25000,5.284770e+08,20.000000,58.000000,7440.250000,5.000000,5.000000,1954.000000,1965.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2007.000000,129500.000000
50%,1465.50000,5.354536e+08,50.000000,68.000000,9436.500000,6.000000,5.000000,1973.000000,1993.000000,0.000000,...,0.000000,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,160000.000000
75%,2197.75000,9.071811e+08,70.000000,80.000000,11555.250000,7.000000,6.000000,2001.000000,2004.000000,164.000000,...,168.000000,70.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,213500.000000
max,2930.00000,1.007100e+09,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,...,1424.000000,742.000000,1012.000000,508.000000,576.000000,800.000000,17000.000000,12.000000,2010.000000,755000.000000


In [5]:
# Get subsets into train & test data
train, test = train_test_split(df, shuffle=True, train_size = 0.7, random_state=42)

# Export train and test data
train.to_csv('exported_train.csv', index=False)
test.to_csv('exported_test.csv', index=False)


train.drop('Order', axis=1)
train.drop('PID', axis=1)



,Order,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
2210,2211,90,RH,82.0,6270,Pave,NA,Reg,HLS,AllPub,...,0,NA,NA,NA,0,8,2007,WD,Normal,145000
782,783,20,RL,65.0,25095,Pave,NA,IR1,Low,AllPub,...,0,NA,NA,NA,0,6,2009,WD,Partial,143000
2310,2311,60,RL,60.0,8238,Pave,NA,IR1,Lvl,AllPub,...,0,NA,NA,NA,0,5,2006,WD,Normal,183500
299,300,120,RM,35.0,3907,Pave,NA,IR1,HLS,AllPub,...,0,NA,NA,NA,0,3,2010,WD,Normal,162500
2423,2424,120,RM,NaN,3072,Pave,NA,Reg,Lvl,AllPub,...,0,NA,NA,NA,0,5,2006,WD,Normal,178740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1638,1639,190,RL,195.0,41600,Pave,NA,IR1,Lvl,AllPub,...,0,NA,NA,NA,0,11,2007,WD,Normal,155000
1095,1096,60,RL,NaN,7750,Pave,NA,Reg,Lvl,AllPub,...,0,NA,NA,NA,0,4,2008,WD,Normal,176000
1130,1131,60,FV,64.0,8791,Pave,NA,IR1,Lvl,AllPub,...,0,NA,NA,NA,0,5,2008,WD,Normal,207500
1294,1295,50,RM,81.0,7308,Pave,NA,Reg,Lvl,AllPub,...,0,NA,NA,NA,0,6,2008,WD,Normal,122250


## Get Training Data back to Tabular data

In [6]:
datastore = ws.get_default_datastore()
datastore.upload_files(files = ['./exported_train.csv'])

train_data = Dataset.Tabular.from_delimited_files([(datastore, 'exported_train.csv')])

Uploading an estimated of 1 files
Uploading ./exported_train.csv
Uploaded ./exported_train.csv, 1 files out of an estimated total of 1
Uploaded 1 files


In [7]:
datastore.upload_files(files=['./exported_test.csv'])
test_data = Dataset.Tabular.from_delimited_files([(datastore, 'exported_test.csv')])

Uploading an estimated of 1 files
Uploading ./exported_test.csv
Uploaded ./exported_test.csv, 1 files out of an estimated total of 1
Uploaded 1 files


## AutoML Configuration



In [10]:
conda_run_configuration = RunConfiguration(framework="python")

# TODO: Put your automl settings here
automl_settings = {
    'name': 'udacity_automl_run',
    'iterations': 20,
    'experiment_timeout_minutes': 20,
    'max_concurrent_iterations': 6,
    'primary_metric': 'normalized_root_mean_squared_error',
    'n_cross_validations': 5,
    'enable_early_stopping': True,
    'featurization': 'auto'
    }

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    task='regression',
    training_data = train_data,
    label_column_name = 'SalePrice',
    debug_log = 'automl_errors.log',
    compute_target='capstone-cluster',
    run_configuration=conda_run_configuration,
    **automl_settings,
    )

In [11]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)


Submitting remote run.
Running on remote compute: capstone-cluster


Experiment,Id,Type,Status,Details Page,Docs Page
predict-housing-prices,AutoML_a6266882-cde6-4422-b33b-66357037a234,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       DONE
DESCRIPTION:  If the missing values are expected, let the run complete. Otherwise cancel the current run and use a script to customize the handling of missing feature values that may be more appropriate based on the data type and business requirement.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization
DETAILS:      
+---------------------------------+---------------------------------+---------------------------------+
|Column name                      |Missing value count              |Imputation type                  |
+===============

In [12]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
predict-housing-prices,AutoML_a6266882-cde6-4422-b33b-66357037a234,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       DONE
DESCRIPTION:  If the missing values are expected, let the run complete. Otherwise cancel the current run and use a script to customize the handling of missing feature values that may be more appropriate based on the data type and business requirement.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization
DETAILS:      
+---------------------------------+---------------------------------+---------------------------------+
|Column name                      |Missing value count              |Imputation type                  |
+=================================+=================================+=================================+
|Lot Frontage                     |347                              |mean                             |
|Mas Vnr Type                  

{'runId': 'AutoML_a6266882-cde6-4422-b33b-66357037a234',
 'target': 'capstone-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-04-19T17:36:04.67574Z',
 'endTimeUtc': '2021-04-19T17:47:01.195988Z',
 'properties': {'num_iterations': '20',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'normalized_root_mean_squared_error',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'capstone-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity_automl_run","subscription_id":"b968fb36-f06a-4c76-a15f-afab68ae7667","resource_group":"aml-quickstarts-143167","workspace_name":"quick-starts-ws-143167","region":"southcentralus","compute_target":"capstone-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"iterations":20,"primary_metric":"normalized_root_mean_squared_error","task_type":"regre

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [12]:
# get run
run = Run(experiment,'AutoML_60263df7-6e97-420e-bbe5-3108f2900075')

# submit script to AML
RunDetails(run).show()
run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

RunId: AutoML_60263df7-6e97-420e-bbe5-3108f2900075
Web View: https://ml.azure.com/runs/AutoML_60263df7-6e97-420e-bbe5-3108f2900075?wsid=/subscriptions/976ee174-3882-4721-b90a-b5fef6b72f24/resourcegroups/aml-quickstarts-143079/workspaces/quick-starts-ws-143079&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Execution Summary
RunId: AutoML_60263df7-6e97-420e-bbe5-3108f2900075
Web View: https://ml.azure.com/runs/AutoML_60263df7-6e97-420e-bbe5-3108f2900075?wsid=/subscriptions/976ee174-3882-4721-b90a-b5fef6b72f24/resourcegroups/aml-quickstarts-143079/workspaces/quick-starts-ws-143079&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



{'runId': 'AutoML_60263df7-6e97-420e-bbe5-3108f2900075',
 'target': 'compute-housing-prices',
 'status': 'Completed',
 'startTimeUtc': '2021-04-18T21:13:19.686206Z',
 'endTimeUtc': '2021-04-18T21:20:52.319652Z',
 'properties': {'num_iterations': '20',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'normalized_root_mean_squared_error',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'compute-housing-prices',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity_automl_run","subscription_id":"976ee174-3882-4721-b90a-b5fef6b72f24","resource_group":"aml-quickstarts-143079","workspace_name":"quick-starts-ws-143079","region":"southcentralus","compute_target":"compute-housing-prices","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"iterations":20,"primary_metric":"normalized_root_mean_squared_error"

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [14]:
# Get the best run as well as the model and print the details.
best_run, model = remote_run.get_output()
print(f"Best Run:\t{best_run}\n\nBest Model:\t{model}")

Best Run:	Run(Experiment: predict-housing-prices,
Id: AutoML_a6266882-cde6-4422-b33b-66357037a234_18,
Type: azureml.scriptrun,
Status: Completed)

Best Model:	RegressionPipeline(pipeline=Pipeline(memory=None,
                                     steps=[('datatransformer',
                                             DataTransformer(enable_dnn=None,
                                                             enable_feature_sweeping=None,
                                                             feature_sweeping_config=None,
                                                             feature_sweeping_timeout=None,
                                                             featurization_config=None,
                                                             force_text_dnn=None,
                                                             is_cross_validation=None,
                                                             is_onnx_compatible=None,
                                

In [15]:
best_run.get_details()

{'runId': 'AutoML_a6266882-cde6-4422-b33b-66357037a234_18',
 'target': 'capstone-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-04-19T17:45:59.453246Z',
 'endTimeUtc': '2021-04-19T17:46:50.197297Z',
 'properties': {'runTemplate': 'automl_child',
  'pipeline_id': '__AutoML_Ensemble__',
  'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'regression\',\'primary_metric\':\'normalized_root_mean_squared_error\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'udacity_automl_run\',\'compute_target\':\'capstone-cluster\',\'subscription_id\':\'b968fb36-f06a-4c76-a15f-afab68ae7667\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_a6266882-cde6-4422-b33b-66357037a234_18","experiment_name":"predict-housing-prices","workspace_name":"quick-starts-ws-143167",

In [14]:
for key, value in best_run.properties.items():
    print(f"{key}\n{value}")

runTemplate
automl_child
pipeline_id
__AutoML_Ensemble__
pipeline_spec
{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{'task_type':'regression','primary_metric':'normalized_root_mean_squared_error','verbosity':20,'ensemble_iterations':15,'is_timeseries':False,'name':'udacity_automl_run','compute_target':'compute-housing-prices','subscription_id':'976ee174-3882-4721-b90a-b5fef6b72f24','region':'southcentralus','spark_service':None}","ensemble_run_id":"AutoML_60263df7-6e97-420e-bbe5-3108f2900075_18","experiment_name":"predict-housing-prices","workspace_name":"quick-starts-ws-143079","subscription_id":"976ee174-3882-4721-b90a-b5fef6b72f24","resource_group_name":"aml-quickstarts-143079"}}]}
training_percent
100
predicted_cost
None
iteration
18
_aml_system_scenario_identification
Remote.Child
_azureml.ComputeTargetType
amlcompute
ContentSnapshotId
c90a

In [16]:
print(best_run.properties['score'])

0.03257659656196975


## Save the best model


In [17]:
model_name = best_run.properties['model_name']

In [19]:
joblib.dump(model, 'best_fit_automl_model.pkl')

['best_fit_automl_model.pkl']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [20]:
# for registering the model
from azureml.core.model import Model
# For generating dependencies for model
from azureml.core.conda_dependencies import CondaDependencies

# Register the model
model = Model.register(workspace=ws, model_name='best_fit_model', model_path='best_fit_automl_model.pkl')



Registering model best_fit_model


In [21]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice

# Create environment
env = best_run.get_environment()
conda_dep = CondaDependencies()

inference_config = InferenceConfig(entry_script='score.py', environment=env)

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=4, enable_app_insights=True)
service = Model.deploy(
    workspace=ws,
    name="predicting-house-prices",
    models=[model],
    inference_config=inference_config,
    deployment_config=deployment_config)



In [22]:
service.wait_for_deployment(show_output = True)

print(service.state)
print(service.scoring_uri)
print(service.swagger_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-04-19 18:02:51+00:00 Creating Container Registry if not exists.
2021-04-19 18:03:02+00:00 Registering the environment.
2021-04-19 18:03:03+00:00 Use the existing image.
2021-04-19 18:03:04+00:00 Generating deployment configuration.
2021-04-19 18:03:05+00:00 Submitting deployment to compute..
2021-04-19 18:03:11+00:00 Checking the status of deployment predicting-house-prices..
2021-04-19 18:08:05+00:00 Checking the status of inference endpoint predicting-house-prices.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
http://260ce883-c9f7-4a0e-9977-0cd8d06f1784.southcentralus.azurecontainer.io/score
http://260ce883-c9f7-4a0e-9977-0cd8d06f1784.southcentralus.azurecontainer.io/swagger.json


TODO: In the cell below, send a request to the web service you deployed to test it.

### Shuffling Test Data to Get an Estimate of the Price for a random Test Row

In [24]:
from sklearn.utils import shuffle

In [25]:
shuffle(test).head(1).drop('SalePrice', axis=1).to_dict('records')

[{'Order': 2252,
  'PID': 914460110,
  'MS SubClass': 20,
  'MS Zoning': 'RL',
  'Lot Frontage': 44.0,
  'Lot Area': 12864,
  'Street': 'Pave',
  'Alley': 'NA',
  'Lot Shape': 'IR1',
  'Land Contour': 'Lvl',
  'Utilities': 'AllPub',
  'Lot Config': 'CulDSac',
  'Land Slope': 'Gtl',
  'Neighborhood': 'Mitchel',
  'Condition 1': 'Norm',
  'Condition 2': 'Norm',
  'Bldg Type': '1Fam',
  'House Style': '1Story',
  'Overall Qual': 7,
  'Overall Cond': 5,
  'Year Built': 2002,
  'Year Remod/Add': 2002,
  'Roof Style': 'Gable',
  'Roof Matl': 'CompShg',
  'Exterior 1st': 'VinylSd',
  'Exterior 2nd': 'VinylSd',
  'Mas Vnr Type': 'None',
  'Mas Vnr Area': 0.0,
  'Exter Qual': 'Gd',
  'Exter Cond': 'TA',
  'Foundation': 'PConc',
  'Bsmt Qual': 'Gd',
  'Bsmt Cond': 'TA',
  'Bsmt Exposure': 'No',
  'BsmtFin Type 1': 'GLQ',
  'BsmtFin SF 1': 1392.0,
  'BsmtFin Type 2': 'Unf',
  'BsmtFin SF 2': 0.0,
  'Bsmt Unf SF': 17.0,
  'Total Bsmt SF': 1409.0,
  'Heating': 'GasA',
  'Heating QC': 'Ex',
  'Centr

In [28]:
# URL for the web service
scoring_uri = 'http://260ce883-c9f7-4a0e-9977-0cd8d06f1784.southcentralus.azurecontainer.io/score'

# Set the content type
headers = {'Content-Type': 'application/json'}


# 
data = {"data":
        shuffle(test).head(1).drop('SalePrice', axis=1).to_dict('records')
        }
# Convert to JSON string
input_data = json.dumps(data)

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)

print("Response Code : ", resp.status_code)
print("Predicted Value : ",resp.text)

Response Code :  200
Predicted Value :  [326411.7933207631]


## Look at Logs of the service

In [29]:
print(service.get_logs())

2021-04-19T18:08:02,057465800+00:00 - rsyslog/run 
2021-04-19T18:08:02,059383200+00:00 - iot-server/run 
2021-04-19T18:08:02,081302600+00:00 - nginx/run 
2021-04-19T18:08:02,080328200+00:00 - gunicorn/run 
/usr/sbin/nginx: /azureml-envs/azureml_1c91e9a5baadb44a18d64bdd7f599d89/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_1c91e9a5baadb44a18d64bdd7f599d89/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_1c91e9a5baadb44a18d64bdd7f599d89/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_1c91e9a5baadb44a18d64bdd7f599d89/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_1c91e9a5baadb44a18d64bdd7f599d89/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

## Delete the Service

In [ ]:
service.delete()